# Важность признаков на основе перестановки

## 1 Импорт библиотек, инициализация глобальных констант

### 1.1 Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
from colorama import Fore, Back, Style
from typing import Tuple, List, Callable, Any
from tqdm import tqdm

from sklearn.utils import check_random_state
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_percentage_error

import lightgbm as lgb
from lightgbm import log_evaluation

warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)

### 1.2 Инициализация констант

In [2]:
TARGET_NAME = 'Цена_квм'
TEST_SIZE = 0.15
RANDOM_STATE = 42

## 2 Подготовка данных

In [3]:
df = pd.read_csv('/kaggle/input/avito-final/houseprices_final.csv')
df.loc[df['Тип_дома'] == 'монолитнокирпичный',
       'Тип_дома'] = 'монолитно-кирпичный'
metros = pd.read_csv('/kaggle/input/avito-final/metro_coords.csv')
df.head()

,ID,Ссылка,Цена,Дата,Адрес,Этаж,Количество_комнат,Балкон_или_лоджия,Тип_комнат,Общая_площадь,Жилая_площадь,Площадь_кухни,Высота_потолков,Санузел,Ремонт,Тип_дома,Год_постройки,Этажей_в_доме,Пассажирский_лифт,Грузовой_лифт,Метро_1,Метро_2,Метро_3,Расстояние_до_метро_1,Расстояние_до_метро_2,Расстояние_до_метро_3,Широта,Долгота,Ипотека,Расстояние_до_Кремля,Окна_на_улицу,Окна_во_двор,Окна_на_солнечную_сторону,Мебель_спальные_места,Мебель_кухня,Мебель_хранение_одежды,В_доме_мусоропровод,В_доме_консьерж,В_доме_газ,Двор_закрытая_территория,Двор_спортивная_площадка,Двор_детская_площадка,Парковка,Количество_лифтов,Цена_квм,Район,Общий_рейтинг,Экология,Чистота,ЖКХ,Соседи,Условия_для_детей,Спорт_и_отдых,Магазины,Транспорт,Безопасность,Стоимость_жизни,Суммарный рейтинг,Общая_площадь_отн_мин,Общая_площадь_минус_мин,Общая_площадь_отн_макс,Общая_площадь_минус_макс,Общая_площадь_отн_сред,Общая_площадь_минус_сред,Расстояние_до_Парк_культуры,Расстояние_до_Октябрьская,Расстояние_до_Добрынинская,Расстояние_до_Павелецкая,Расстояние_до_Таганская,Расстояние_до_Красные_ворота,Расстояние_до_Проспект_Мира,Расстояние_до_Курская,Расстояние_до_Белорусская,Расстояние_до_Новослободская,Расстояние_до_Киевская,Расстояние_до_Краснопресненская,Общая_площадь_кластер,Координаты_кластер
0,2636072348,https://avito.ru//moskva/kvartiry/3-k._kvartir...,33000000,2022-12-02,"Москва, Ленинградское шоссе, 69к2",18.0,3,балкон,изолированные,73.2,NaN,16.0,NaN,NaN,дизайнерский,монолитный,NaN,20.0,0,0,Беломорская,Речной вокзал,Планерная,30.0,30.0,31.0,55.868713,37.456819,0,16.417191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,450819.672131,Левобережный,39.0,39.0,84.0,65.0,38.0,44.0,40.0,102.0,81.0,17.0,23.0,572.0,1.763855,31.7,0.281538,-186.8,0.873901,-10.562326,17.083389,18.258180,18.672102,19.183695,18.657365,16.339909,14.839579,17.596692,12.816081,13.400078,15.492292,14.198571,8,40
1,2626697262,https://avito.ru//moskva/kvartiry/3-k._kvartir...,39000000,2022-12-03,"Москва, Ходынская улица, 2",13.0,3,NaN,изолированные,73.7,50.0,15.0,NaN,совмещенный,дизайнерский,монолитный,NaN,44.0,0,0,Улица 1905 года,Беговая,Белорусская,1905.0,20.0,20.0,55.770949,37.563930,0,3.971249,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,529172.320217,Пресненский,62.0,93.0,45.0,67.0,16.0,85.0,52.0,109.0,40.0,10.0,79.0,658.0,1.775904,32.2,0.283462,-186.3,0.879871,-10.062326,4.329917,5.493617,5.934142,6.548749,6.432229,5.254251,4.464546,6.107069,1.346540,2.524840,3.094803,1.435916,8,124
2,2658691692,https://avito.ru//moskva/kvartiry/3-k._kvartir...,22500000,2022-12-05,"Москва, Аминьевское ш., 4Дк3литБ",9.0,3,лоджия,изолированные,69.3,43.9,5.7,3.0,раздельный,требует ремонта,монолитный,2022.0,28.0,2,2,Аминьевская,Давыдково,Мичуринский проспект,15.0,30.0,31.0,55.698791,37.468364,1,11.067722,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,1.0,4,324675.324675,Очаково-Матвеевское,93.0,76.0,62.0,62.0,76.0,88.0,112.0,96.0,107.0,77.0,94.0,943.0,1.669880,27.8,0.266538,-190.7,0.827341,-14.462326,8.955781,9.558359,10.224293,11.210438,12.558108,13.636461,13.702383,13.664497,11.277326,12.245417,7.763103,9.657269,8,11
3,2515435010,https://avito.ru//moskva/kvartiry/3-k._kvartir...,17900000,2022-12-05,"Москва, Севастопольский пр-т, 13к2",5.0,3,лоджия,изолированные,62.5,45.0,7.0,2.8,совмещенный,евро,панельный,1979.0,9.0,1,0,Крымская,Верхние Котлы,Нагатинская,15.0,20.0,20.0,55.683567,37.602962,0,7.665333,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1,286400.000000,Котловка,53.0,36.0,22.0,53.0,78.0,62.0,71.0,37.0,16.0,47.0,113.0,588.0,1.506024,21.0,0.240385,-197.5,0.746159,-21.262326,5.869496,5.106472,5.197940,5.601104,7.262720,9.832084,10.848071,9.048621,10.519448,10.679586,7.054251,8.693040,0,10
4,1972516369,https://avito.ru//moskva/kvartiry/10_i_bolee-k...,525000000,2022-12-05,"Москва, Пресненская набережная, 6с2",59.0,10 и больше,NaN,изолированные,941.7,NaN,20.0,NaN,NaN,косметический,монолитный,2010.0,60.0,1,0,Деловой центр,Выставочная,Международная,5.0,5.0,10.0,55.748071,37.540397,0,4.859495,Na

### 2.1 Кодирование станций метро

In [4]:
metro_stations = np.append(metros['Станция'].values, ['нет'])

le = LabelEncoder()

le.fit(metro_stations.reshape(-1, 1))
for i in range(1, 4):
    df[f'Метро_{i}'] = le.transform(df[f'Метро_{i}'])

### 2.2 Кодирование категориальных переменных

In [5]:
cat_features = [
    'Количество_комнат', 'Балкон_или_лоджия', 'Тип_комнат', 'Санузел',
    'Ремонт', 'Тип_дома', 'Район', 'Ипотека', 'Окна_во_двор',
    'Окна_на_солнечную_сторону', 'Окна_на_улицу', 'Мебель_спальные_места',
    'Мебель_кухня', 'Мебель_хранение_одежды', 'В_доме_консьерж',
    'В_доме_мусоропровод', 'В_доме_газ', 'Двор_детская_площадка',
    'Двор_закрытая_территория', 'Двор_спортивная_площадка', 'Парковка'
]

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
df[cat_features] = encoder.fit_transform(df[cat_features])

### 2.3 Отложенная выборка

In [6]:
train, test = train_test_split(df,
                               test_size=TEST_SIZE,
                               stratify=df['Количество_комнат'],
                               random_state=RANDOM_STATE)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

## 3 Вспомогательные функции

### 3.1 Функция для обучения модели

In [7]:
def lgb_training(train_df, params, features, test_df=None, verbose=250):
    """
    Функция для обучения модели.
    
    Параметры
    ----------
    train_df - обучающая выборка
    params - набор гиперпараметров
    features - признаки
    test_df - тестовая выборка
    verbose - вывод результатов
    
    Возвращает
    -------
    score_list - MAPE на кросс-валидации для каждого фолда
    test_preds - предсказания на тестовой выборке, если тестовая выборка задана
    oof_preds - предсказания для обучающей выборки
    """
    test_preds = []
    if test_df is not None:
        test_preds = np.zeros(len(test_df))
        test_df = test_df[features]
    target = train_df.loc[:, TARGET_NAME].values

    categorical_feature = list(set(features) & set(cat_features))
    score_list = []
    oof_preds = np.zeros(len(target))

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
        X_train = train_df.loc[train_index, features]
        X_val = train_df.loc[val_index, features]
        y_train = target[train_index]
        y_val = target[val_index]

        lgb_train = lgb.Dataset(X_train,
                                y_train,
                                categorical_feature=categorical_feature)
        lgb_valid = lgb.Dataset(X_val,
                                y_val,
                                categorical_feature=categorical_feature)

        callbacks = [lgb.log_evaluation(period=verbose)]
        model = lgb.train(params=params,
                          train_set=lgb_train,
                          num_boost_round=15000,
                          valid_sets=[lgb_train, lgb_valid],
                          categorical_feature=categorical_feature,
                          callbacks=callbacks)

        preds = model.predict(X_val)
        oof_preds[val_index] = preds
        score = mean_absolute_percentage_error(y_val, preds)

        if test_df is not None:
            test_preds += model.predict(test_df) / kf.n_splits

        if verbose:
            print(
                f"{Fore.BLACK}{Style.BRIGHT}FOLD: {fold}, MAPE: {score:.5f} {Style.RESET_ALL}"
            )
        score_list.append(score)

    return score_list, test_preds, oof_preds

### 3.2 Функция для получения permutation importance
[Источник](https://www.kaggle.com/code/simakov/keras-multilabel-neural-network-v1-2)

In [8]:
def iter_shuffled(X,
                  columns_to_shuffle=None,
                  pre_shuffle=False,
                  random_state=None):
    rng = check_random_state(random_state)

    if columns_to_shuffle is None:
        columns_to_shuffle = range(X.shape[1])

    if pre_shuffle:
        X_shuffled = X.copy()
        rng.shuffle(X_shuffled)

    X_res = X.copy()
    for columns in columns_to_shuffle:
        if pre_shuffle:
            X_res[:, columns] = X_shuffled[:, columns]
        else:
            rng.shuffle(X_res[:, columns])
        yield X_res
        X_res[:, columns] = X[:, columns]


def get_score_importances(score_func,
                          X,
                          y,
                          n_iter=5,
                          columns_to_shuffle=None,
                          random_state=None):
    rng = check_random_state(random_state)
    base_score = score_func(X, y)
    scores_decreases = []
    for i in trange(n_iter):
        scores_shuffled = _get_scores_shufled(
            score_func,
            X,
            y,
            columns_to_shuffle=columns_to_shuffle,
            random_state=rng,
            base_score=base_score)
        scores_decreases.append(scores_shuffled)

    return base_score, scores_decreases


def _get_scores_shufled(score_func,
                        X,
                        y,
                        base_score,
                        columns_to_shuffle=None,
                        random_state=None):
    Xs = iter_shuffled(X, columns_to_shuffle, random_state=random_state)
    res = []
    for X_shuffled in Xs:
        res.append(-score_func(X_shuffled, y) + base_score)
    return res

## 4 Отбор признаков

In [9]:
features_to_drop = ['ID', 'Ссылка', 'Цена', 'Дата', 'Адрес', 'Цена_квм']
features = [f for f in df.columns if f not in features_to_drop]

In [10]:
params = {
    'objective': 'mape',
    'learning_rate': 0.3,
    'reg_lambda': 5,
    'random_state': 42,
    'early_stopping_rounds': 300,
    'feature_fraction': 0.4,
    'verbose': -1
}

In [11]:
perm_imp = np.zeros(len(features))
all_res = []
perm_df = pd.DataFrame({'feature': features})
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_index, val_index) in enumerate(kf.split(train)):
    print(f'Fold {fold}')

    X_train = train.loc[train_index, features]
    X_val = train.loc[val_index, features]
    y_train = train.loc[train_index, TARGET_NAME]
    y_val = train.loc[val_index, TARGET_NAME]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_valid = lgb.Dataset(X_val, y_val)

    callbacks = [lgb.log_evaluation(period=0)]
    model = lgb.train(params=params,
                      train_set=lgb_train,
                      num_boost_round=15000,
                      valid_sets=[lgb_train, lgb_valid],
                      categorical_feature=cat_features,
                      callbacks=callbacks)

    def _score(X, y):
        pred = model.predict(X)
        return mean_absolute_percentage_error(y, pred)

    base_score, local_imp = get_score_importances(_score,
                                                  X_val.values,
                                                  y_val.values,
                                                  n_iter=25,
                                                  random_state=RANDOM_STATE)
    all_res.append(local_imp)
    perm_imp += np.mean(local_imp, axis=0)
    perm_df[f"fold_{fold}_importance"] = perm_imp
    print('')

Fold 0


100%|██████████| 25/25 [08:37<00:00, 20.71s/it]



Fold 1


100%|██████████| 25/25 [18:34<00:00, 44.59s/it]



Fold 2


100%|██████████| 25/25 [27:31<00:00, 66.07s/it]



Fold 3


100%|██████████| 25/25 [36:48<00:00, 88.32s/it]



Fold 4


100%|██████████| 25/25 [16:14<00:00, 38.98s/it]

## 5 Результаты

In [12]:
perm_df['importance'] = perm_df.filter(regex='^fold').mean(axis=1)
perm_df = perm_df.sort_values(by='importance')
best_feats = perm_df.sort_values(by='importance')['feature'].values
perm_df[perm_df['importance'] < 0]

,feature,fold_0_importance,fold_1_importance,fold_2_importance,fold_3_importance,fold_4_importance,importance
39,Район,-0.072606,-0.155049,-0.230344,-0.316836,-0.397648,-0.234497
9,Ремонт,-0.019788,-0.038830,-0.059197,-0.081384,-0.101081,-0.060056
12,Этажей_в_доме,-0.012171,-0.025392,-0.039578,-0.055331,-0.068704,-0.040235
69,Расстояние_до_Краснопресненская,-0.008042,-0.021744,-0.039967,-0.047869,-0.058259,-0.035176
11,Год_постройки,-0.011889,-0.022864,-0.034603,-0.044560,-0.058231,-0.034429
10,Тип_дома,-0.008222,-0.018191,-0.029477,-0.040608,-0.050773,-0.029454
24,Расстояние_до_Кремля,-0.010699,-0.018962,-0.022290,-0.033554,-0.042598,-0.025620
68,Расстояние_до_Киевская,-0.004054,-0.011340,-0.021483,-0.033489,-0.044497,-0.022973
53,Общая_площадь_минус_мин,-0.006324,-0.013349,-0.019309,-0.025965,-0.032894,-0.019568
5,Жилая_площадь,-0.006271,-0.011906,-0.019352,-0.024299,-0.032072,-0.018780


Самым важным признаком оказался район, его перемешивание увеличивает ошибку в среднем на 0.23

Попробуем найти лучшую комбинацию признаков в количестве менее 35 штук

In [13]:
result_df = pd.DataFrame(columns=['features', 'score'])
for i in trange(5, 36):
    current_feats = best_feats[:i]
    score_list, _, _ = lgb_training(train, params, current_feats, verbose=0)
    result_df = result_df.append(
        {
            'features': current_feats,
            'score': np.mean(score_list)
        },
        ignore_index=True)

100%|██████████| 31/31 [1:45:07<00:00, 203.48s/it]


In [14]:
result_df = result_df.sort_values(by='score')
display(result_df.head(10))
display(result_df.head(10)['features'].values)

,features,score
29,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.123427
30,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.124134
25,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.124179
28,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.124258
27,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.124532
22,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.124864
21,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.125106
24,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.125502
16,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.125641
23,"[Район, Ремонт, Этажей_в_доме, Расстояние_до_К...",0.125690


array([array(['Район', 'Ремонт', 'Этажей_в_доме',
              'Расстояние_до_Краснопресненская', 'Год_постройки', 'Тип_дома',
              'Расстояние_до_Кремля', 'Расстояние_до_Киевская',
              'Общая_площадь_минус_мин', 'Жилая_площадь', 'Этаж',
              'Площадь_кухни', 'Общая_площадь', 'Расстояние_до_Парк_культуры',
              'Высота_потолков', 'Расстояние_до_метро_1',
              'Двор_закрытая_территория', 'Санузел', 'Расстояние_до_Курская',
              'Общая_площадь_минус_макс', 'Расстояние_до_Белорусская',
              'Расстояние_до_Добрынинская', 'Расстояние_до_Таганская',
              'Расстояние_до_Проспект_Мира', 'Расстояние_до_Красные_ворота',
              'Долгота', 'Ипотека', 'В_доме_газ', 'Расстояние_до_Новослободская',
              'Расстояние_до_Октябрьская', 'Общая_площадь_минус_сред',
              'Мебель_кухня', 'В_доме_консьерж', 'Балкон_или_лоджия'],
             dtype=object)                                                       ,
 